In [6]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import warnings
import pandas as pd
import pickle

from tqdm import tqdm
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from pathlib import Path
base_path = os.path.join(os.path.dirname(os.getcwd()))
print(f"Base path: {base_path}")
data_path = Path(base_path, "data", "bepro", "processed")
yaml_file = Path(base_path, "assertion", "transitions.yaml")

from assertion.bepro import convert_to_actions
from assertion.validator import Validator

Base path: /root/express-v2


In [5]:
match_id_lst = os.listdir(data_path)
match_id_lst

['126424',
 '126364',
 '126332',
 '153387',
 '126386',
 '126306',
 '126380',
 '126401',
 '126356',
 '126433',
 '126458',
 '126476',
 '126309',
 '126341',
 '153390',
 '126473',
 '153364',
 '126391',
 '126444',
 '153379',
 '126350',
 '126315',
 '126285']

In [11]:
match_id = match_id_lst[0]  # Change this to the desired match ID
with open(f"{data_path}/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
    match_dict = pickle.load(f)
    events = match_dict['event_df']
    teams_dict = match_dict['teams']
    metadata = match_dict['meta_data']

pd.concat([events.groupby("period_order").head(1), 
           events.groupby("period_order").tail(1)])

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction
0,Half,1st Half,0,2700000,0,533,Gwangju FC,99,Beka Mikeltadze,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4976,0.5045,0.4236,0.6365,RIGHT
0,Half,2nd Half,1,2700000,2700000,2700300,Daegu FC,17,Italo Carvalho,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4977,0.5119,0.4088,0.5073,RIGHT
1600,Half,1st Half,0,2700000,0,2879667,Gwangju FC,11,Gabriel,"[{'event_name': 'Fouls', 'property': {'Type': ...",0.5844,0.0887,NaN,NaN,RIGHT
1836,Half,2nd Half,1,2700000,2700000,6004233,Gwangju FC,22,Sanggi Lee,"[{'event_name': 'Duels', 'property': {'Type': ...",0.9684,0.6135,NaN,NaN,LEFT


In [13]:
teams_df = pd.concat([teams_dict["Home"], teams_dict["Away"]], ignore_index=True)
player_name_to_player_id = {row.player: int(row.pID) for row in teams_df.itertuples()}
player_id_to_team_id = {int(row.pID): int(row.tID) for row in teams_df.itertuples()}
player_id_to_team = {int(row.pID): row.team for row in teams_df.itertuples()}

events["game_id"] = match_id
events["event_id"] = range(len(events))
events["player_id"] = events["player_name"].map(player_name_to_player_id).astype(int)
events["team_id"] = events["player_id"].map(player_id_to_team_id).astype(int)
events["team"] = events["player_id"].map(player_id_to_team)
events

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,...,y,to_x,to_y,attack_direction,game_id,event_id,player_id,team_id,team,period_id
0,Half,1st Half,0,2700000,0,533,Gwangju FC,99,Beka Mikeltadze,"[{'event_name': 'Passes', 'property': {'Outcom...",...,0.5045,0.4236,0.6365,RIGHT,126424,0,408383,4648,Away,1
1,Half,1st Half,0,2700000,0,1667,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",...,0.6365,NaN,NaN,RIGHT,126424,1,77414,4648,Away,1
2,Half,1st Half,0,2700000,0,5167,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",...,0.6543,0.3413,0.7579,RIGHT,126424,2,77414,4648,Away,1
3,Half,1st Half,0,2700000,0,6500,Gwangju FC,18,Yool Heo,"[{'event_name': 'Passes Received', 'property':...",...,0.7579,NaN,NaN,RIGHT,126424,3,145703,4648,Away,1
4,Half,1st Half,0,2700000,0,15021,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 500...",...,0.5752,0.3501,0.5339,RIGHT,126424,4,145703,4648,Away,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,Half,2nd Half,1,2700000,2700000,5999333,Daegu FC,11,Cesinha,"[{'event_name': 'Passes Received', 'property':...",...,0.1069,NaN,NaN,RIGHT,126424,3433,62112,4644,Home,2
1833,Half,2nd Half,1,2700000,2700000,6000600,Daegu FC,11,Cesinha,"[{'event_name': 'Passes', 'property': {'Outcom...",...,0.1376,0.7890,0.2495,RIGHT,126424,3434,62112,4644,Home,2
1834,Half,2nd Half,1,2700000,2700000,6002367,Daegu FC,2,Jaewon Hwang,"[{'event_name': 'Crosses', 'property': {'Outco...",...,0.2495,0.9670,0.6118,RIGHT,126424,3435,250055,4644,Home,2
1835,Half,2nd Half,1,2700000,2700000,6004200,Daegu FC,17,Italo Carvalho,"[{'event_name': 'Duels', 'property': {'Type': ...",...,0.6118,NaN,NaN,RIGHT,126424,3436,530662,4644,Home,2


In [14]:
actions = convert_to_actions(events)
actions

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,...,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy
0,126424.0,0,0.0,1.0,0.533,1.667,4648.0,408383.0,77414.0,-1.0,...,1.0,-1.0,52.2480,34.3060,44.4780,43.2820,44.4780,43.2820,-7.7700,8.9760
1,126424.0,1,1.0,1.0,1.667,NaN,4648.0,77414.0,-1.0,-1.0,...,-1.0,-1.0,44.4780,43.2820,44.4780,43.2820,NaN,NaN,0.0000,0.0000
2,126424.0,2,2.0,1.0,5.167,6.500,4648.0,77414.0,145703.0,-1.0,...,3.0,-1.0,46.8720,44.4924,35.8365,51.5372,35.8365,51.5372,-11.0355,7.0448
3,126424.0,3,3.0,1.0,6.500,NaN,4648.0,145703.0,-1.0,-1.0,...,-1.0,-1.0,35.8365,51.5372,35.8365,51.5372,NaN,NaN,0.0000,0.0000
4,126424.0,5,4.0,1.0,15.933,17.733,4648.0,145703.0,250100.0,-1.0,...,6.0,-1.0,36.4350,34.0476,34.4505,6.5008,34.4505,6.5008,-1.9845,-27.5468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,126424.0,3435,2299.0,2.0,3302.367,NaN,4644.0,250055.0,-1.0,-1.0,...,-1.0,-1.0,22.1550,51.0340,3.4650,26.3976,NaN,NaN,-18.6900,-24.6364
2300,126424.0,3435,2300.0,2.0,3302.367,NaN,4644.0,250055.0,-1.0,-1.0,...,-1.0,-1.0,22.1550,51.0340,3.4650,26.3976,NaN,NaN,-18.6900,-24.6364
2301,126424.0,3436,2301.0,2.0,3304.200,NaN,4644.0,530662.0,-1.0,-1.0,...,-1.0,3437.0,3.4650,26.3976,3.4650,26.3976,NaN,NaN,0.0000,0.0000
2302,126424.0,3437,2302.0,2.0,3304.233,NaN,4648.0,250100.0,-1.0,-1.0,...,-1.0,3436.0,101.6820,41.7180,101.6820,41.7180,NaN,NaN,0.0000,0.0000


In [15]:
validator = Validator(actions, yaml_file)
validator.validate_sequence()
validator.df_events

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,...,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy
0,126424.0,0,0,1.0,0.533,1.667,4648.0,408383.0,77414.0,-1.0,...,1.0,-1.0,52.2480,34.3060,44.4780,43.2820,44.4780,43.2820,-7.7700,8.9760
1,126424.0,1,1,1.0,1.667,NaN,4648.0,77414.0,-1.0,-1.0,...,-1.0,-1.0,44.4780,43.2820,44.4780,43.2820,NaN,NaN,0.0000,0.0000
2,126424.0,2,2,1.0,5.167,6.500,4648.0,77414.0,145703.0,-1.0,...,3.0,-1.0,46.8720,44.4924,35.8365,51.5372,35.8365,51.5372,-11.0355,7.0448
3,126424.0,3,3,1.0,6.500,NaN,4648.0,145703.0,-1.0,-1.0,...,-1.0,-1.0,35.8365,51.5372,35.8365,51.5372,NaN,NaN,0.0000,0.0000
4,126424.0,NaN,4,1.0,6.501,NaN,4648.0,145703.0,NaN,-1.0,...,NaN,NaN,35.8365,51.5372,36.4350,34.0476,NaN,NaN,0.5985,-17.4896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,126424.0,3435,2851,2.0,3302.367,NaN,4644.0,250055.0,-1.0,-1.0,...,-1.0,-1.0,22.1550,51.0340,3.4650,26.3976,NaN,NaN,-18.6900,-24.6364
2852,126424.0,3435,2852,2.0,3302.367,NaN,4644.0,250055.0,-1.0,-1.0,...,-1.0,-1.0,22.1550,51.0340,101.6820,41.7180,NaN,NaN,79.5270,-9.3160
2853,126424.0,3437,2853,2.0,3304.233,NaN,4648.0,250100.0,NaN,4644.0,...,NaN,NaN,101.6820,41.7180,101.6820,41.7180,NaN,NaN,0.0000,0.0000
2854,126424.0,NaN,2854,2.0,3304.233,NaN,4648.0,250100.0,NaN,-1.0,...,NaN,NaN,101.6820,41.7180,101.6820,41.7180,NaN,NaN,0.0000,0.0000


In [16]:
validator.df_errors

,game_id,event_id,current_state,next_state,event,error
0,126424.0,3.0,in_possession,in_possession,Carry,Dribble Add
1,126424.0,9.0,in_possession,in_possession,Carry,Dribble Add
2,126424.0,11.0,in_possession,in_possession,Carry,Dribble Add
3,126424.0,23.0,in_possession,in_possession,Carry,Dribble Add
4,126424.0,30.0,in_possession,in_possession,Carry,Dribble Add
...,...,...,...,...,...,...
613,126424.0,3395.0,in_possession,in_possession,Carry,Dribble Add
614,126424.0,3405.0,in_possession,in_possession,Carry,Dribble Add
615,126424.0,3415.0,in_possession,in_possession,Carry,Dribble Add
616,126424.0,NaN,ball_neutral,set_piece,Out,Resolved


In [21]:
validator.df_events["type_name"].unique()

array(['Pass', 'Pass Received', 'Carry', 'Foul', 'Pass_Freekick',
       'Tackle', 'Clearance', 'Out', 'Throw-In', 'Duel', 'Recovery',
       'Interception', 'Intervention', 'Cross', 'Block', 'Pass_Corner',
       'Offside', 'Shot', 'Aerial Clearance', 'Parry', 'Take-On',
       'Goal Kick', 'Defensive Line Support', 'Catch', 'Shot_Freekick',
       'Goal', 'Cross Received', 'Ball Received', 'Error', 'Goal Post'],
      dtype=object)